<a href="https://colab.research.google.com/github/gmazzitelli/studenti/blob/master/esempio1_fit_popolazioneIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![metrodo scintifico](https://3.bp.blogspot.com/-srz1XJ37dE8/VgwypJp1xAI/AAAAAAAAJG0/BDSVcwHW2MQ/s1600/Metodo-Scientifico01-mini.jpg)
**Introduzione**

1.   ossevazione: raccolta dai dati (decisione, quali e come);
2.   ipotesi: modello matematico (decisione);
3.   verifica: contronto del modello con i dati;
4.   predizione: quando, dove, come...

**Strumento**

1.   notebook, jupyter, colab: https://colab.research.google.com/
2.   python (root, C, LabView...): https://www.python.org/




In [0]:
#
# inzializzazione delle funzioni (librerie) usate nel notebook
#
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
from scipy.stats import chisquare
plt.rc('font', size = 14, family ='Sans-Serif')

In [0]:
# caricamento dati raccolti (popolazione italiana 2018)
url_csv_file = "https://docs.google.com/spreadsheet/ccc?key=1qL5B_J6eV6bD9zicR0YsCIaUMU0BhTdSTrqC1UcDBI8&output=csv"
df = pd.read_csv(url_csv_file)
print(df)
print('\n')
print ('Popolazione: {0:.1f} [M], Superficie: {1:.1f} [km^2], Densità: {2:.1f}'.format(
    df['Popolazione'].sum()/1000, df['Superficie'].sum()/1000, df['Popolazione'].sum()/df['Superficie'].sum()))

In [0]:
#
# Visualizziamo i dati
#
fig, ax = plt.subplots(figsize=(10,6))
x = df['Superficie']/1000
y = df['Popolazione']/1000
ax.plot(x,y,'ro')
# ax.set_xlim(0,30)
# ax.set_ylim(0,12)
# ax.set_xlabel('Superficie [$km^2$]')
# ax.set_ylabel(r'Popolazione [$\times 10^6$]')
# plt.title('Densità Popolazione Italiana')
# for k in range(x.size):
#   plt.annotate(df['Regione'][k], (x[k],y[k]),fontsize=12)
# plt.minorticks_on()
# plt.grid()
# plt.tick_params(axis="both", direction='in', which="both", length=6, width=1, colors='k',
#                grid_color='grey', grid_alpha=0.5)
plt.show()

In [0]:
#
# definisco il modello (lineare) 
#
def Line(x, m, q):
    import numpy as np
    return m*x + q

In [0]:
#
# esguiamo il fit con la funzione che abbiamo scelto come modello con i minimi quadrati
# https://it.wikipedia.org/wiki/Metodo_dei_minimi_quadrati
# 
par, cov = curve_fit(Line,x, y, p0=[1,0])
err  = np.sqrt(np.diag(cov))
r2   = r2_score(y , Line(x, *par)) # https://it.wikipedia.org/wiki/Coefficiente_di_determinazione
chi2 = chisquare(y, Line(x, *par)) # https://it.wikipedia.org/wiki/Test_chi_quadrato
print(r'm = {0:.2f}±{1:.2f} q = {2:.1f}±{3:.1f} r2 = {4:.4f} chi2 = {5:.2f}'.format(
    par[0], err[0], par[1], err[1], r2, chi2[0]))

In [0]:
#
# Visualizziamo il risulatato
#
fig, ax = plt.subplots(figsize=(10,6))
ax.plot(x,y,'ro', label='dati regioni')
ax.plot(x,Line(x, *par), 'k-', 
        label='m = {0:.2f}±{1:.2f} \nq = {2:.1f}±{3:.1f} \nr2 = {4:.4f}'.format(par[0], err[0], par[1], err[1], r2))
ax.set_xlim(0,30)
ax.set_ylim(0,12)
ax.set_xlabel('Superficie [$km^2$]')
ax.set_ylabel(r'Popolazione [$\times 10^6$]')
plt.title('Densità Popolazione Italiana')
plt.minorticks_on()
plt.grid()
plt.tick_params(axis="both", direction='in', which="both", length=6, width=1, colors='k',
               grid_color='grey', grid_alpha=0.5)
plt.legend()
plt.show()

In [0]:
fig, ax = plt.subplots(2,2,figsize=(15,10))
x = df['Superficie']/1000
y = df['Comuni']
ax[0,0].set_xlabel('Superficie [$km^2$]')
ax[0,0].set_ylabel('Comuni')
ax[0,0].plot(x,y,'ro')
x = df['Superficie']/1000
y = df['Province']
ax[0,1].set_xlabel('Superficie [$km^2$]')
ax[0,1].set_ylabel('Province')
ax[0,1].plot(x,y,'ro')
x = df['Popolazione']/1000
y = df['Comuni']
ax[1,0].set_xlabel(r'Popolazione [$\times 10^6$]')
ax[1,0].set_ylabel('Comuni')
ax[1,0].plot(x,y,'ro')
x = df['Popolazione']/1000
y = df['Province']
ax[1,1].set_xlabel(r'Popolazione [$\times 10^6$]')
ax[1,1].set_ylabel('Province')
ax[1,1].plot(x,y,'ro')
plt.show()

**Precauzioni!**

esitono vari modi di fare i fit, e anche in questo e' necessario decidere!
![alt text](https://camo.githubusercontent.com/732f5f5631f683f7da87f3489d4767ff6afe2315/687474703a2f2f6372736f757a612e636f6d2f77702d636f6e74656e742f75706c6f6164732f323031302f30362f72616e736163385f7468756d622e706e67)

i minimi quadrati, e' il metoto piu' comune ma puo' sbagliare 
per questo si usano anche altri metodi che ottimizziano cio' che cerchimao come ad esempio RANSAC https://scikit-learn.org/stable/auto_examples/linear_model/plot_ransac.html e piu' in generale il Machine Leaning

In [0]:
# def Gauss3(x, a0, x0, s0):
    import numpy as np
    return a0 * np.exp(-(x - x0)**2 / (2 * s0**2))

def test_chi2(y, fy, stdev):
    r = y - fy
    chi2 = np.sum((r/stdev)**2)
    return chi2
#
# chi2, p = chisquare(y, function(x), gradi)
#